In [ ]:
# Step by steps to generate data
# 1. Generate prompt embeddings
# 2. Generate images from prompts
# 3. Generate correlation between prompt and image

In [ ]:
prompts = [
    'hyper realistic photo of very friendly and dystopian crater',
    'ramen carved out of fractal rose ebony, in the style of hudson river school',
    'ultrasaurus holding a black bean taco in the woods, near an identical cheneosaurus',
    'a thundering retro robot crane inks on parchment with a droopy french bulldog',
    'portrait painting of a shimmering greek hero, next to a loud frill-necked lizard',
    'an astronaut standing on a engaging white rose, in the midst of by ivory cherry blossoms',
    'Kaggle employee Phil at a donut shop ordering all the best donuts, with a speech bubble that proclaims "Donuts. It\'s what\'s for dinner!"',
]

In [ ]:
# !pip install sentence_transformers

In [ ]:
# generate embeddings using minilm-l6-v2
from sentence_transformers import SentenceTransformer, models
st_model = SentenceTransformer('/Users/thanh/Desktop/works/kaggle/sditp/data/all-MiniLM-L6-v2')

In [ ]:
embeddings = []

for prompt in prompts:
    prompt_embeddings = st_model.encode(prompt).flatten()
    embeddings.append(prompt_embeddings)

In [ ]:
embeddings[0].shape

In [ ]:
# generate images from prompts

In [ ]:
# !pip install --upgrade diffusers transformers -q 

In [ ]:
# !pip install -U huggingface_hub

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

In [ ]:
class CFG:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    seed = 42
    generator = torch.Generator(device).manual_seed(seed)
    model_id = "stabilityai/stable-diffusion-2"

CFG = CFG()

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# TODO: need to check if below code is similar to this: https://github.com/CompVis/stable-diffusion/blob/main/scripts/txt2img.py

model_pipe = StableDiffusionPipeline.from_pretrained(CFG.model_id, torch_dtype=torch.float16, revision="fp16", use_auth_token=True)
model_pipe = model_pipe.to(CFG.device)

In [ ]:
def generate_image(prompt, model):
    return model(prompt).images[0]

In [ ]:
generated_images = []
images_idx = []
for idx, prompt in enumerate(prompts):
    generated_images.append(generate_image(prompt, model_pipe))
    images_idx.append(idx)

In [ ]:
import os

images_path = "./images"
if not os.path.exists(images_path):
    os.mkdir(images_path)

for idx, image in zip(images_idx, generated_images):
    image.save(images_path / f"{str(idx).zfill(5)}.png")

In [ ]:
prompts_df = pd.DataFrame({"imgId": [str(idx).zfill(5) for idx in images_idx], "prompt": prompts})
prompts_df["imgId"] = prompts_df["imgId"].astype("str")
prompts_df